In [10]:
import numpy as np
import torch
from torch import nn

In [2]:
data = np.load("data/images.npy")
labels = np.load("data/labels.npy")

In [9]:
data.shape

(18000, 150, 150)

In [13]:
18000/4


4500.0

In [11]:
class NN_regression(nn.Module):
    """ Convolution model that returns one value as output.
    """
    def __init__(self, input_channels):
        self.input_layer = nn.Sequential(
            nn.Conv2d(input_channels, 32, kernel_size=5),
        )
        self.hidden_layers = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten()
        )
        self.output_layer = nn.Sequential(
            nn.Linear(32323, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )